In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
# Load the Fama-French 3 Factor data (assuming you've downloaded the CSV)
ff_5_data = pd.read_csv('/data/workspace_files/lazy_price_replication/F-F_Research_Data_5_Factors_2x3_daily.CSV', skiprows=3, skipfooter = 1)

# Rename columns for clarity
ff_5_data.columns = ['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW','CMA','RF']

# Convert the Date column to datetime
ff_5_data['Date'] = pd.to_datetime(ff_5_data['Date'], format='%Y%m%d')

# Filter data for the period 2008-2012
ff_5_filtered = ff_5_data[(ff_5_data['Date'] >= '2008-01-01') & (ff_5_data['Date'] <= '2012-12-31')]
ff_5_filtered = ff_5_filtered.set_index('Date')

<ipython-input-3-82864357d168>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  ff_5_data = pd.read_csv('/data/workspace_files/lazy_price_replication/F-F_Research_Data_5_Factors_2x3_daily.CSV', skiprows=3, skipfooter = 1)


In [7]:
#load portfolio returns
portfolio_rets = pd.read_csv('/data/workspace_files/lazy_price_replication/jaccard_sim_returns')
portfolio_rets['daily_return'] = portfolio_rets['daily_return']*100
portfolio_rets['Date'] = pd.to_datetime(portfolio_rets['Date'])
portfolio_rets.set_index('Date')

,daily_return
Date,
2008-01-02,NaN
2008-01-03,-0.685723
2008-01-04,-2.549489
2008-01-07,-0.231512
2008-01-08,-1.318033
...,...
2011-12-23,0.294098
2011-12-27,0.335985
2011-12-28,-1.736874


In [8]:
#merging the data
merged_data = pd.merge(portfolio_rets,ff_5_filtered,on='Date')
merged_data = merged_data.dropna()
merged_data

,Date,daily_return,Mkt-RF,SMB,HML,RMW,CMA,RF
1,2008-01-03,-0.685723,-0.14,-1.08,-0.41,-0.14,-0.34,0.01
2,2008-01-04,-2.549489,-2.56,-0.50,0.05,0.05,0.12,0.01
3,2008-01-07,-0.231512,0.16,0.13,0.18,0.38,0.98,0.01
4,2008-01-08,-1.318033,-1.81,-0.59,-0.94,0.24,-0.01,0.01
5,2008-01-09,0.068085,1.10,-0.39,-0.66,0.68,0.08,0.01
...,...,...,...,...,...,...,...,...
1004,2011-12-23,0.294098,0.84,-0.57,-0.12,0.30,-0.03,0.00
1005,2011-12-27,0.335985,0.04,0.47,-0.55,0.41,-0.24,0.00
1006,2011-12-28,-1.736874,-1.34,-0.70,-0.08,0.24,0.25,0.00
1007,2011-12-29,0.989436,1.10,0.24,0.29,-0.20,0.09,0.00


In [9]:
#regression against FAMA French factors

# Step 1: Calculate the excess portfolio return
merged_data['excess_portfolio_return'] = merged_data['daily_return'] - merged_data['RF']

# Step 2: Define the independent variables (Fama-French factors)
X = merged_data[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]

# Step 3: Add a constant (intercept) for the regression
X = sm.add_constant(X)

# Step 4: Define the dependent variable (excess portfolio return)
y = merged_data['excess_portfolio_return']

# Step 5: Perform the OLS regression
model = sm.OLS(y, X).fit()

# Step 6: Get the results, including the alpha (intercept)
print(model.summary())

                               OLS Regression Results                              
Dep. Variable:     excess_portfolio_return   R-squared:                       0.750
Model:                                 OLS   Adj. R-squared:                  0.749
Method:                      Least Squares   F-statistic:                     599.8
Date:                     Mon, 07 Oct 2024   Prob (F-statistic):          8.19e-298
Time:                             03:35:57   Log-Likelihood:                -1503.9
No. Observations:                     1005   AIC:                             3020.
Df Residuals:                          999   BIC:                             3049.
Df Model:                                5                                         
Covariance Type:                 nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
co